In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
from torch.autograd import Variable

In [4]:
from IPython.display import clear_output
!pip install transformers
!pip install datasets
clear_output()

In [11]:
from transformers import AutoTokenizer
from datasets import load_dataset

#Prepare model

In [85]:
class TextGRU(nn.Module):
    '''
    Recurrent Neural Network based on Gated Rectified Unit for text classification [1].
    Credits for implementation: [2]
    Inputs:
        vocab_size - size of vocabulary of dataset used
        emb_dim - embedding space dimension
        hidden_dim - dimension of hidden state of RNN
        output_dim - dimension of the output (number of classes in our case)
        dropout - drop out probability
        pad_idx - padding index for sequences
    [1]: Empirical evaluation of gated recurrent neural networks on sequence modeling, 2014.
    [2]: https://github.com/etomoscow/DL-in-NLP/blob/master/hw3/task3_sentiment_rnn.ipynb
    '''

    def __init__(self, vocab_size, emb_dim, hidden_dim, out_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.GRU(emb_dim, hidden_dim, num_layers=1, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted=False)
        packed_output, hidden = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        return self.fc(hidden)


# Prepare dataset


In [86]:
#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [87]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
train_dataset = load_dataset('trec', split='train')
train_dataset = train_dataset.map(
    lambda e: tokenizer(e['text'], truncation=True, padding='max_length'),
    batched=True
    )
test_dataset = load_dataset('trec', split='test')
test_dataset = test_dataset.map(
    lambda e: tokenizer(e['text'], truncation=True, padding='max_length'),
    batched=True
    )
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

Using custom data configuration default
Reusing dataset trec (/root/.cache/huggingface/datasets/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48)
Loading cached processed dataset at /root/.cache/huggingface/datasets/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48/cache-efaae888953b3ce0.arrow
Using custom data configuration default
Reusing dataset trec (/root/.cache/huggingface/datasets/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48)
Loading cached processed dataset at /root/.cache/huggingface/datasets/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48/cache-3f3d932597ca390c.arrow


In [89]:
model = TextGRU(
    vocab_size=tokenizer.vocab_size,
    emb_dim=100,
    hidden_dim=128,
    out_dim=6,
    dropout=0.1,
    pad_idx=tokenizer.pad_token_id)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [109]:
def trec_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim=1, keepdim=True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    correct = correct.detach().to('cpu')
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [110]:
def get_text_length(batch, tokenizer):
    result = []
    for i in range(batch.shape[1]):
        result.append((sum(batch[:, i] != tokenizer.pad_token_id).item()))
    return torch.tensor(result, dtype=int, device='cpu')

In [111]:
get_text_length(torch.stack(x['input_ids'], dim=0), tokenizer)

tensor([14, 11, 14, 17, 11, 15, 14,  8,  7, 14, 10, 10, 10,  6, 16, 14, 18, 13,
        12, 10, 10, 12,  7,  9, 13, 15, 11, 12, 10, 11, 10, 10, 14, 19, 12,  8,
        11, 11, 18, 18, 13, 17, 11, 14, 13, 12, 18, 10, 14, 13, 13,  8, 11, 15,
        12,  9, 14, 12, 13,  8,  7, 21, 10,  6])

In [116]:
def train(model, iterator, optimizer, criterion, tokenizer):
    global device
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        batch_ = torch.stack(batch['input_ids'], dim=0)
        batch_ = batch_.to(device)
        optimizer.zero_grad()
        text_lengths = get_text_length(batch_, tokenizer) 
        predictions = model(batch_, text_lengths)
        loss = criterion(predictions, batch['label-coarse'].long().to(device))
        acc = trec_accuracy(predictions, batch['label-coarse'].to(device))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [117]:
def evaluate(model, iterator, criterion, tokenizer):
    global device
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            batch_ = torch.stack(batch['input_ids'], dim=0)
            batch_ = batch_.to(device)
            text_lengths = get_text_length(batch_, tokenizer)
            predictions = model(batch_, text_lengths)
            loss = criterion(predictions, batch['label-coarse'].long().to(device))
            acc = trec_accuracy(predictions, batch['label-coarse'].to(device))
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [118]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [119]:
N_EPOCHS = 50
best_valid_loss = float('inf')
losses = []
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, trainloader, optimizer, criterion, tokenizer)
    valid_loss, valid_acc = evaluate(model, testloader, criterion, tokenizer)
    losses.append(train_loss)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'textrnn_trec.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    if epoch >= 3:
        if train_loss >= losses[-1] and train_loss >= losses[-2] and train_loss >= losses[-3]:
            print('Early stopping')
            break 

Epoch: 01 | Epoch Time: 0m 38s
	Train Loss: 1.182 | Train Acc: 53.94%
	 Val. Loss: 0.820 |  Val. Acc: 72.94%
Epoch: 02 | Epoch Time: 0m 39s
	Train Loss: 0.744 | Train Acc: 71.92%
	 Val. Loss: 0.626 |  Val. Acc: 77.19%
Epoch: 03 | Epoch Time: 0m 40s
	Train Loss: 0.541 | Train Acc: 80.47%
	 Val. Loss: 0.596 |  Val. Acc: 78.67%
Epoch: 04 | Epoch Time: 0m 40s
	Train Loss: 0.387 | Train Acc: 86.70%
	 Val. Loss: 0.616 |  Val. Acc: 77.94%
Epoch: 05 | Epoch Time: 0m 41s
	Train Loss: 0.274 | Train Acc: 91.09%
	 Val. Loss: 0.714 |  Val. Acc: 79.45%
Epoch: 06 | Epoch Time: 0m 41s
	Train Loss: 0.182 | Train Acc: 94.04%
	 Val. Loss: 0.849 |  Val. Acc: 78.03%
Epoch: 07 | Epoch Time: 0m 40s
	Train Loss: 0.123 | Train Acc: 96.26%
	 Val. Loss: 0.807 |  Val. Acc: 79.64%
Epoch: 08 | Epoch Time: 0m 40s
	Train Loss: 0.080 | Train Acc: 97.33%
	 Val. Loss: 0.672 |  Val. Acc: 82.42%
Epoch: 09 | Epoch Time: 0m 39s
	Train Loss: 0.052 | Train Acc: 98.62%
	 Val. Loss: 0.730 |  Val. Acc: 82.68%
Epoch: 10 | Epoch T

In [120]:
model.eval()

TextGRU(
  (embedding): Embedding(30522, 100, padding_idx=0)
  (rnn): GRU(100, 128, bidirectional=True)
  (fc): Linear(in_features=256, out_features=6, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)